# Train Custom LLM from Scratch

This notebook trains a language model from scratch with configurable:
- **Model size** (125M to 7B parameters)
- **Model type** (Reasoning Agent, Code Assistant, General Purpose, etc.)
- **Dataset selection** (automatically matched to model type)

## Requirements
- **GPU**: A100 40GB+ (80GB recommended for 3B+)
- **Storage**: Google Drive for checkpoints
- **Time**: Varies by model size and type

---
## Step 0: Environment Setup

In [ ]:
#@title ### 0.1 Mount Google Drive & Clone Repository
import os
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted")
else:
    print("Running locally")

In [ ]:
#@title ### 0.2 Clone Repository & Install Dependencies
REPO_URL = "https://github.com/rmarnold/llm-training-pipeline.git"  #@param {type:"string"}
BRANCH = "main"  #@param {type:"string"}

REPO_DIR = "/content/llm-training-pipeline"

if IN_COLAB:
    if os.path.exists(REPO_DIR):
        %cd {REPO_DIR}
        !git pull origin {BRANCH}
    else:
        !git clone -b {BRANCH} {REPO_URL} {REPO_DIR}
        %cd {REPO_DIR}
    
    print("\nInstalling dependencies...")
    !pip install -q -e ".[colab]"
    !pip install -q flash-attn --no-build-isolation 2>/dev/null || true
    !pip install -q liger-kernel bitsandbytes
    
    # Install GPU acceleration libraries (RAPIDS + NeMo Curator)
    print("\nInstalling GPU acceleration libraries...")
    !pip install -q --extra-index-url=https://pypi.nvidia.com cudf-cu12 cuml-cu12 2>/dev/null || echo "RAPIDS not available"
    !pip install -q "nemo-curator[text_cuda12]" 2>/dev/null || echo "NeMo Curator not available"
    
    PROJECT_ROOT = REPO_DIR
else:
    PROJECT_ROOT = os.getcwd()

os.chdir(PROJECT_ROOT)
print(f"\nProject root: {PROJECT_ROOT}")

In [ ]:
#@title ### 0.3 Choose Model Type & Size { run: "auto" }
#@markdown ---
#@markdown ### Model Configuration

model_type = "reasoning_agent"  #@param ["reasoning_agent", "code_assistant", "general_assistant", "chat_model"]
#@markdown **Model Types:**
#@markdown - `reasoning_agent`: Math, logic, function calling, tool use
#@markdown - `code_assistant`: Code generation, debugging, explanation
#@markdown - `general_assistant`: Balanced instruction following
#@markdown - `chat_model`: Conversational, helpful responses

model_size = "1b"  #@param ["125m", "350m", "1b", "3b", "7b"]
#@markdown **Model Sizes:**
#@markdown - `125m`: Fast training, testing (~2-4 hours pretrain)
#@markdown - `350m`: Small but capable (~4-8 hours)
#@markdown - `1b`: Good balance (~40-50 hours)
#@markdown - `3b`: Strong performance (~80-100 hours)
#@markdown - `7b`: Full capability (~150+ hours)

#@markdown ---
#@markdown ### Training Parameters
context_length = 2048  #@param {type:"integer"}
pretrain_tokens_billions = 50  #@param {type:"number"}
#@markdown *Recommended: 20x model params (1B model = 20B tokens minimum)*

#@markdown ---
#@markdown ### Data Preparation Settings
data_prep_speed = "fast"  #@param ["fast", "balanced", "thorough"]
#@markdown **Data Prep Quality Modes:**
#@markdown - `fast`: Skip toxicity filter, basic quality only (~30 min for 12M docs)
#@markdown - `balanced`: Basic quality + toxicity filter (~2-3 hours)
#@markdown - `thorough`: All quality filters + toxicity (~6-7 hours)

use_gpu_data_prep = True  #@param {type:"boolean"}
#@markdown **GPU Data Prep (Recommended for A100/H100):**
#@markdown - `True`: Use RAPIDS cuDF + NeMo Curator (10-30x faster)
#@markdown - `False`: Use CPU-only pipeline (slower but more compatible)

# ============================================================
# MODEL TYPE CONFIGURATIONS
# ============================================================

MODEL_TYPE_CONFIGS = {
    "reasoning_agent": {
        "description": "Optimized for math, logic, and tool use",
        "pretraining_datasets": [
            "slimpajama", "wikipedia", "the-stack-python", 
            "openwebtext", "arxiv", "stackexchange"
        ],
        "sft_datasets": [
            "gsm8k", "orca-math", "openorca", "cot-collection", "metamath",
            "glaive-function-calling", "hermes-function-calling", "toolbench",
            "logiqa"
        ],
        "dpo_datasets": ["hh-rlhf", "ultrafeedback"],
        "data_prep_script": "06_prepare_reasoning_data.py",
        "eval_benchmarks": ["gsm8k", "arc", "function_calling", "safety"],
        "sft_focus": "reasoning",
    },
    "code_assistant": {
        "description": "Optimized for code generation and understanding",
        "pretraining_datasets": [
            "the-stack", "slimpajama", "wikipedia", "arxiv"
        ],
        "sft_datasets": [
            "code-alpaca", "python-code-instructions", "evol-instruct-code",
            "glaive-function-calling", "oasst1"
        ],
        "dpo_datasets": ["hh-rlhf"],
        "data_prep_script": "prepare_lora_data.py",  # Reuse code prep
        "eval_benchmarks": ["humaneval", "mbpp", "safety"],
        "sft_focus": "code",
    },
    "general_assistant": {
        "description": "Balanced instruction following",
        "pretraining_datasets": [
            "slimpajama", "wikipedia", "openwebtext", "pg19"
        ],
        "sft_datasets": [
            "oasst1", "dolly-15k", "alpaca-cleaned", "openorca"
        ],
        "dpo_datasets": ["hh-rlhf", "ultrafeedback"],
        "data_prep_script": "06_prepare_sft_data.py",
        "eval_benchmarks": ["mmlu", "hellaswag", "safety"],
        "sft_focus": "instruction",
    },
    "chat_model": {
        "description": "Conversational and helpful",
        "pretraining_datasets": [
            "slimpajama", "wikipedia", "openwebtext"
        ],
        "sft_datasets": [
            "oasst1", "dolly-15k", "sharegpt"
        ],
        "dpo_datasets": ["hh-rlhf", "ultrafeedback"],
        "data_prep_script": "06_prepare_sft_data.py",
        "eval_benchmarks": ["mt-bench", "safety"],
        "sft_focus": "chat",
    },
}

# Data prep speed configurations (CPU pipeline)
DATA_PREP_CONFIGS = {
    "fast": {
        "flags": "--native-pipeline --fast-quality --no-toxicity --fresh",
        "gpu_flags": "--fast-quality --no-toxicity",
        "description": "Fastest: GopherQuality only, no toxicity filter",
    },
    "balanced": {
        "flags": "--native-pipeline --fast-quality --fresh",
        "gpu_flags": "--fast-quality",
        "description": "Balanced: GopherQuality + toxicity filter",
    },
    "thorough": {
        "flags": "--native-pipeline --fresh",
        "gpu_flags": "",
        "description": "Thorough: All quality filters + toxicity",
    },
}

# Size to training time estimates (A100 80GB)
SIZE_CONFIGS = {
    "125m": {"pretrain_hours": 4, "sft_hours": 1, "dpo_hours": 0.5, "batch_size": 32, "grad_accum": 1},
    "350m": {"pretrain_hours": 8, "sft_hours": 2, "dpo_hours": 1, "batch_size": 16, "grad_accum": 2},
    "1b": {"pretrain_hours": 50, "sft_hours": 12, "dpo_hours": 4, "batch_size": 16, "grad_accum": 4},
    "3b": {"pretrain_hours": 100, "sft_hours": 24, "dpo_hours": 8, "batch_size": 8, "grad_accum": 8},
    "7b": {"pretrain_hours": 200, "sft_hours": 48, "dpo_hours": 16, "batch_size": 8, "grad_accum": 4},
}

# Build configuration
type_config = MODEL_TYPE_CONFIGS[model_type]
size_config = SIZE_CONFIGS[model_size]
data_prep_config = DATA_PREP_CONFIGS[data_prep_speed]

CONFIG = {
    'model_type': model_type,
    'model_size': model_size,
    'context_length': context_length,
    'pretrain_tokens_b': pretrain_tokens_billions,
    'batch_size': size_config['batch_size'],
    'grad_accum': size_config['grad_accum'],
    'pretraining_datasets': type_config['pretraining_datasets'],
    'sft_datasets': type_config['sft_datasets'],
    'dpo_datasets': type_config['dpo_datasets'],
    'data_prep_script': type_config['data_prep_script'],
    'eval_benchmarks': type_config['eval_benchmarks'],
    'sft_focus': type_config['sft_focus'],
    'data_prep_flags': data_prep_config['flags'],
    'gpu_data_prep_flags': data_prep_config['gpu_flags'],
    'data_prep_speed': data_prep_speed,
    'use_gpu_data_prep': use_gpu_data_prep,
}

# Calculate training steps
tokens_per_step = CONFIG['batch_size'] * CONFIG['grad_accum'] * context_length
total_tokens = int(pretrain_tokens_billions * 1e9)
pretrain_steps = total_tokens // tokens_per_step
CONFIG['pretrain_steps'] = pretrain_steps

# Estimate time
time_scale = pretrain_tokens_billions / 50  # Base estimate is for 50B tokens
pretrain_hours = size_config['pretrain_hours'] * time_scale
total_hours = pretrain_hours + size_config['sft_hours'] + size_config['dpo_hours']

# Drive path based on model type
DRIVE_BASE = f"/content/drive/MyDrive/llm-{model_size}-{model_type.replace('_', '-')}"

print("=" * 60)
print("MODEL CONFIGURATION")
print("=" * 60)
print(f"\nType: {model_type.upper().replace('_', ' ')}")
print(f"  {type_config['description']}")
print(f"\nSize: {model_size.upper()}")
print(f"  Context: {context_length} tokens")
print(f"  Batch: {CONFIG['batch_size']} x {CONFIG['grad_accum']} = {CONFIG['batch_size'] * CONFIG['grad_accum']} effective")
print(f"\nPretraining:")
print(f"  Tokens: {pretrain_tokens_billions}B")
print(f"  Steps: {pretrain_steps:,}")
print(f"\nData Preparation: {data_prep_speed.upper()}")
print(f"  {data_prep_config['description']}")
print(f"  GPU Acceleration: {'ENABLED (10-30x faster)' if use_gpu_data_prep else 'DISABLED'}")
print(f"\nDatasets:")
print(f"  Pretraining: {', '.join(type_config['pretraining_datasets'][:4])}...")
print(f"  SFT: {', '.join(type_config['sft_datasets'][:4])}...")
print(f"\nEstimated Time (A100):")
print(f"  Pretraining: ~{pretrain_hours:.0f} hours")
print(f"  SFT: ~{size_config['sft_hours']} hours")
print(f"  DPO: ~{size_config['dpo_hours']} hours")
print(f"  Total: ~{total_hours:.0f} hours ({total_hours/24:.1f} days)")
print(f"\nStorage: {DRIVE_BASE}")
print("=" * 60)

In [ ]:
#@title ### 0.4 Set Up Persistent Storage

if IN_COLAB:
    print(f"Setting up storage at: {DRIVE_BASE}")
    
    # Create Drive directories
    for subdir in ['checkpoints', 'data', 'data/raw', 'data/packed', 'data/sft', 'data/dpo', 'logs', 'evals']:
        os.makedirs(os.path.join(DRIVE_BASE, subdir), exist_ok=True)
    
    # Create symlinks
    for dir_name in ['checkpoints', 'data', 'logs', 'evals']:
        local_path = os.path.join(PROJECT_ROOT, dir_name)
        drive_path = os.path.join(DRIVE_BASE, dir_name)
        
        if os.path.exists(local_path) and not os.path.islink(local_path):
            !cp -r {local_path}/* {drive_path}/ 2>/dev/null || true
            !rm -rf {local_path}
        elif os.path.islink(local_path):
            os.unlink(local_path)
        
        os.symlink(drive_path, local_path)
        print(f"  {dir_name} -> Drive")
    
    print("\nStorage ready!")
else:
    for d in ['checkpoints', 'data', 'logs', 'evals']:
        os.makedirs(d, exist_ok=True)

In [ ]:
#@title ### 0.5 Check GPU
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    capability = torch.cuda.get_device_capability()
    
    CONFIG['use_fp8'] = capability[0] >= 9
    
    print(f"GPU: {gpu_name} ({gpu_memory:.0f} GB)")
    print(f"FP8: {'Available' if CONFIG['use_fp8'] else 'Not available'}")
    
    # Memory check for model size
    min_memory = {'125m': 8, '350m': 16, '1b': 24, '3b': 40, '7b': 70}
    if gpu_memory < min_memory[CONFIG['model_size']]:
        print(f"\nWARNING: {CONFIG['model_size']} may need {min_memory[CONFIG['model_size']]}+ GB")
else:
    print("No GPU detected!")
    CONFIG['use_fp8'] = False

---
## Step 1: Download Data

Downloads datasets matched to your model type.

In [ ]:
#@title ### 1.1 Download Pretraining Data
#@markdown Downloads pretraining corpora for your model type.

print(f"Downloading pretraining data for: {CONFIG['model_type']}")
print(f"Datasets: {CONFIG['pretraining_datasets']}")
print("="*50)

!python scripts/01_download_data.py --phases pretraining

In [ ]:
#@title ### 1.2 Download SFT & DPO Data
#@markdown Downloads fine-tuning data matched to your model type.

print(f"Downloading SFT data for: {CONFIG['model_type']}")
print(f"SFT datasets: {CONFIG['sft_datasets']}")
print(f"DPO datasets: {CONFIG['dpo_datasets']}")
print("="*50)

# Download based on model type
if CONFIG['sft_focus'] == 'reasoning':
    !python scripts/01_download_data.py --phases reasoning function_calling logic instruction_tuning preference_data
elif CONFIG['sft_focus'] == 'code':
    !python scripts/01_download_data.py --phases instruction_tuning preference_data
    # Code data handled by prepare_lora_data.py
else:
    !python scripts/01_download_data.py --phases instruction_tuning preference_data

---
## Step 2: Prepare Data

In [ ]:
#@title ### 2.1 Clean & Tokenize Pretraining Data
#@markdown Uses the data prep speed setting from configuration above.
#@markdown GPU mode uses RAPIDS cuDF (150x faster text cleaning) + NeMo Curator (16x faster dedup).

use_gpu = CONFIG.get('use_gpu_data_prep', False)
data_prep_speed = CONFIG['data_prep_speed']

print(f"Data prep mode: {data_prep_speed.upper()}")
print(f"GPU Acceleration: {'ENABLED' if use_gpu else 'DISABLED'}")
print("=" * 50)

if use_gpu:
    # GPU-accelerated pipeline (10-30x faster on A100)
    gpu_flags = CONFIG['gpu_data_prep_flags']
    print("Using GPU pipeline (RAPIDS cuDF + NeMo Curator)...")
    !python scripts/02_gpu_clean_deduplicate.py --input data/raw --output data/processed {gpu_flags}
else:
    # CPU pipeline (original)
    data_prep_flags = CONFIG['data_prep_flags']
    print(f"Using CPU pipeline with flags: {data_prep_flags}")
    !python scripts/02_clean_deduplicate_optimized.py {data_prep_flags}

# Tokenize and pack (same for both)
!python scripts/03_tokenize_and_pack.py

In [ ]:
#@title ### 2.2 Prepare SFT Data
#@markdown Uses the appropriate script for your model type.

prep_script = CONFIG['data_prep_script']
print(f"Using: {prep_script} (optimized for {CONFIG['model_type']})")
print("="*50)

!python scripts/{prep_script}

In [ ]:
#@title ### 2.3 Prepare DPO Data

!python scripts/08_prepare_dpo_data.py

---
## Step 3: Initialize Model

In [ ]:
#@title ### 3.1 Initialize Model

model_size = CONFIG['model_size']
context = CONFIG['context_length']

print(f"Initializing {model_size.upper()} model with {context} context...")

!python scripts/04_init_model.py --size {model_size} --context-length {context}

In [ ]:
#@title ### 3.2 Verify Setup
import os

checks = [
    ('Model', 'checkpoints/init'),
    ('Tokenizer', 'configs/tokenizer'),
    ('Pretrain data', 'data/packed'),
    ('SFT data', 'data/sft/train'),
    ('DPO data', 'data/dpo/train'),
]

print("Setup verification:")
all_ok = True
for name, path in checks:
    ok = os.path.exists(path)
    print(f"  {name}: {'OK' if ok else 'MISSING'}")
    all_ok = all_ok and ok

if all_ok:
    print("\nReady for training!")
else:
    print("\nFix missing items before proceeding.")

---
## Step 4: Pretraining

In [ ]:
#@title ### 4.1 Start Pretraining

steps = CONFIG['pretrain_steps']
use_fp8 = CONFIG.get('use_fp8', False)

cmd = f"python scripts/05_pretrain.py --max_steps {steps}"
cmd += " --use-liger-kernel --use-cce --enable-oom-recovery"
if use_fp8:
    cmd += " --fp8"

print(f"Model: {CONFIG['model_size'].upper()} {CONFIG['model_type']}")
print(f"Steps: {steps:,}")
print(f"Command: {cmd}")
print("="*50)

!{cmd}

In [ ]:
#@title ### 4.2 Resume (if interrupted)
# CHECKPOINT = "checkpoints/pretrain/checkpoint-10000"
# !python scripts/05_pretrain.py --resume_from_checkpoint {CHECKPOINT}

---
## Step 5: SFT Training

In [ ]:
#@title ### 5.1 Start SFT

cmd = "python scripts/07_sft.py --use-liger-kernel --enable-oom-recovery"
if CONFIG.get('use_fp8', False):
    cmd += " --fp8"

print(f"SFT focus: {CONFIG['sft_focus']}")
print(f"Datasets: {CONFIG['sft_datasets']}")
print("="*50)

!{cmd}

---
## Step 6: DPO Alignment

In [ ]:
#@title ### 6.1 Start DPO

cmd = "python scripts/09_dpo.py --enable-oom-recovery"
if CONFIG.get('use_fp8', False):
    cmd += " --fp8"

!{cmd}

---
## Step 7: Evaluation

In [ ]:
#@title ### 7.1 Run Evaluation

print(f"Benchmarks for {CONFIG['model_type']}: {CONFIG['eval_benchmarks']}")
print("="*50)

!python scripts/11_evaluate.py checkpoints/dpo_final

In [ ]:
#@title ### 7.2 Check Gates

!python scripts/12_check_gates.py dpo

---
## Step 8: Test Model

In [ ]:
#@title ### 8.1 Load Model
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_PATH = "checkpoints/dpo_final"

tokenizer = AutoTokenizer.from_pretrained("configs/tokenizer")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print("Model loaded!")

In [ ]:
#@title ### 8.2 Generate Response

def generate(prompt, max_tokens=512):
    formatted = f"<|user|>\n{prompt}\n<|assistant|>\n"
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_tokens, 
                                 temperature=0.7, do_sample=True, top_p=0.9)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test prompts based on model type
TEST_PROMPTS = {
    'reasoning_agent': "Solve step by step: If a train travels 60 mph for 2.5 hours, how far does it go?",
    'code_assistant': "Write a Python function to find the nth Fibonacci number.",
    'general_assistant': "Explain the difference between machine learning and deep learning.",
    'chat_model': "Hello! How are you today? What can you help me with?",
}

prompt = TEST_PROMPTS.get(CONFIG['model_type'], "Hello, how are you?")
print(f"Test for {CONFIG['model_type']}:")
print(f"Prompt: {prompt}")
print("\nResponse:")
print(generate(prompt))

In [ ]:
#@title ### 8.3 Custom Prompt

CUSTOM_PROMPT = """Your prompt here"""  #@param {type:"string"}

print(generate(CUSTOM_PROMPT))

---
## Training Complete!

Your model is trained and ready to use. Check the evaluation results above to see how it performs on relevant benchmarks.